# PPE Object Detection ( Using Yolo7 )
**CapStone Project**
## ITAI-2277-Artificial Intel Resource.
**Professor: G. Raymond Brown**
### Team Members:
1. Mohamed Mohamed.
2. Sumesh Surendran.
3. Cheyenne Hathaway.
4. Bryanna Wallace


### Introduction :
we will walk  through the process of training an object detection model that can detect people wearing a helmet. This project consists of below phases:
1. Install YOLOv7 dependencies.
2. Preparing dataset.
3. Annotation converting from PASCAL VOC format to YOLO.
5. Setup the YAML files for training
6. Training the model
7. Testing the model in images and videos


## Step 1:
Install YOLOv7 dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# cloning of the yolov7 official repository:
!git clone https://github.com/WongKinYiu/yolov7

In [ ]:
%cd /content/drive/MyDrive/mido/yolov7

/content/drive/MyDrive/mido/yolov7


In [ ]:
# installing all the requirements libraries:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00


In [ ]:
# Download the trained weights for yolo7:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

## Step 2:
Preparing your own dataset:
- we used the MakeMl Dataset ( we can use any other dataset from any resource but make sure that the annotations in the yolo7 format) .
- The dataset includes images of the following class:
1. Helmet
2. Person
3. Head.

In [ ]:
# creating a directory in the same folder with yolo7 :
import os

if not os.path.isdir('DataSet'):
  os.makedirs('DataSet')

In [ ]:
%cd DataSet

In [ ]:
# downloading the dataset:
!wget -O hardhatworkerspascalvoc.zip https://arcraftimages.s3-accelerate.amazonaws.com/Datasets/HardHatWorkers/HardHatWorkersPascalVOC.zip?region=us-east-2

In [ ]:
# unzip the dataset:
unzip hardhatworkerspascalvoc.zip

## Step 3:
Converting annotations from PASCAL-VOC to YOLOv7


In [ ]:
# This is a little script that does it for you:
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"helmet": 0,
                           "person": 1,
                           "head": 2}

def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()

    # Initialise the info dict
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name
        if elem.tag == "filename":
            info_dict['filename'] = elem.text

        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))

            info_dict['image_size'] = tuple(image_size)

        # Get details of the bounding box
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text

                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)
            info_dict['bboxes'].append(bbox)

    return info_dict


# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []

    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])

        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width    /= image_w
        b_height   /= image_h

        #Write the bbox details to the file
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))

    # Name of the file which we have to save
    save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))

    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

### your project directory should be like :
yolov7       
└─── Dataset        
    │
    └─── images        
            │
            └─── test       
            │
            └─── train       
            │
            └─── val       
    └──  labels       
            │
            └─── test       
            │
            └─── train       
            │
            └─── val       

In [ ]:
# split dataset:
import os
from sklearn.model_selection import train_test_split
import shutil
# Read images and annotations
images = [os.path.join('images', x) for x in os.listdir('images')]
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'newimages/train')
move_files_to_folder(val_images, 'newimages/val/')
move_files_to_folder(test_images, 'newimages/test/')
move_files_to_folder(train_annotations, 'newannotations/train/')
move_files_to_folder(val_annotations, 'newannotations/val/')
move_files_to_folder(test_annotations, 'newannotations/test/')

## Step 4:
Setup the YAML files for training:     

- create a new YAML file with this data:     
train: /home/jetson/yolov7/dataset/images/train/   #(train images path)     
val:  /home/jetson/yolov7/dataset/images/val/      #(validating images path)     
test: /home/jetson/yolov7/dataset/images/test/      #(test images path)     
- number of classes     
nc: 3     
-  class names     
names: ["helmet","person", "head"]

## step 5:
Training the model:     
**Notes before training:**
1. Make sure your data YAML file has the right bath for your data.
2. change the number of classes in the cfg YAML file to 3.


In [ ]:
!python train.py --device 0 --batch-size 16 --epochs 15 --img 640 640 --data data/coco.yaml --hyp data/hyp.scratch.custom.yaml --cfg cfg/training/yolov7x.yaml --weights "yolov7x (1).pt" --name yolov7x_custom

2023-11-20 22:10:53.375459: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 22:10:53.375513: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 22:10:53.375538: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 22:10:53.383563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 22:10:54.333682: W tensorflow/compiler/

## step 6:
Testing the model in images and videos

In [ ]:
!python3 detect.py --weights "runs/train/yolov7x_custom2/weights/best.pt"  --conf 0.5 --img-size 640 --source "1.jpg"

Namespace(weights=['runs/train/yolov7x_custom2/weights/best.pt'], source='1.jpg', img_size=640, conf_thres=0.5, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.1.0+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
IDetect.fuse
Model Summary: 362 layers, 70816134 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
1 Helmet, 2 Persons, 2 Vests, Done. (36.3ms) Inference, (1.6ms) NMS
 The image with the result is saved

## step 7:
Evaluating the Model

In [ ]:
!python test.py --weights best.pt --task test --data data/custom_data.yaml

Namespace(weights=['best.pt'], data='data/custom_data.yaml', batch_size=32, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.1.0+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70795920 parameters, 0 gradients, 188.0 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 


Traceback (most recent call last):
  File "/content/drive/MyDrive/mido/yo

## step 8:
Running the model on real time camera streaming

In [ ]:
!python3 detect.py --weights "runs/train/yolov7x_custom2/weights/best.pt"  --conf 0.5 --img-size 640 --source 0 --no-trace

## Resources:
1. https://www.hackster.io/shahizat/safety-helmet-detection-system-based-on-yolov7-algorithm-3d4cef
2. https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection (another link for the dataset)
3. https://blog.paperspace.com/train-yolov5-custom-data/
4. https://www.youtube.com/watch?v=XzUMigbYRUI